In [1]:
import sys
import os
import yaml
import random
from aip import AipSpeech
root_dir = "/xiaobai/"
sys.path.append(root_dir)
from xiaobai import XiaoBai,BaseSkill

In [14]:
from bluepy.btle import Scanner, DefaultDelegate

class ScanDelegate(DefaultDelegate):
    def __init__(self):
        DefaultDelegate.__init__(self)

    def handleDiscovery(self, dev, isNewDev, isNewData):
        if isNewDev:
            pass
            #print("Discovered device ",  dev.addr)
        elif isNewData:
            pass
            #print("Received new data from", dev.addr)

def find_dev(name=""):
    scanner = Scanner().withDelegate(ScanDelegate())
    devices = scanner.scan(10.0)

    for dev in devices:
        #print("Device %s (%s), RSSI=%d dB" % (dev.addr, dev.addrType, dev.rssi) )
        for (adtype, desc, value) in dev.getScanData():
            if desc == "Complete Local Name" and value==name:
                print("find MJ_HT_V1")
                return dev.addr
    return None
if __name__ == '__main__':
    print(find_dev('MJ_HT_V1'))

find MJ_HT_V1
58:2d:34:30:36:d1


In [11]:
from bluepy import btle
import re

class XiomiHygroThermoDelegate(object):
    def __init__(self):
        self.temperature = None
        self.humidity = None
        self.received = False
        self.count = 0

    def handleNotification(self, cHandle, data):
        #print(data)
        if cHandle == 14:
            self.count = self.count + 1
            if self.count % 2 == 1 : 
                return
            else:
                m = re.search('T=([\d\.]*)\s+?H=([\d\.]*)', ''.join(map(chr, data)))
                self.temperature = m.group(1)
                self.humidity = m.group(2)
                #print("温度："+self.temperature+"℃，湿度："+self.humidity+"%")
                self.received = True
            
class SensorSkill(BaseSkill):
    def __init__(self):
        self.address = "58:2d:34:30:36:d1"
    #继承BaseSkill类，必须定义has_intent和action方法
    def has_intent(self,text=""):
        for keyword in ["温度","湿度","温湿度","气温"]:
            if text.find(keyword)>= 0:
                return True
        return False
    def action(self,text="",callback=print):
        try:
            p = btle.Peripheral(self.address)
            delegate = XiomiHygroThermoDelegate()
            p.withDelegate( delegate )
            p.writeCharacteristic(0x10, bytearray([1, 0]), True)
            while not delegate.received:
                p.waitForNotifications(4.0)
            temperature = delegate.temperature
            humidity = delegate.humidity
            callback("温度："+temperature+"℃，湿度："+humidity+"%")
        except:
            callback("出错了呢")
if __name__ == '__main__':
    import time
    s = SensorSkill()
    s.handle("温度是多少")

温度：27.9℃，湿度：18.9%


In [ ]:
keyword_model = root_dir+'resources/小白.pmdl'
xiaobai = XiaoBai(keyword_model=keyword_model)
xiaobai.add_skill(SensorSkill())
xiaobai.listen_for_keyword()

Listening...


INFO:snowboy:Keyword 1 detected at time: 2019-05-13 14:47:15


你：温度湿度
小白：温度：28.5℃，湿度：19.1%
